<div><img src="logo.gif" width="220" ALIGN="left" border="20"></div>
<center><font color="4466ff"> <h1>AutoCV Starting Kit </h1> </font></center>
<br>
<p>This code was tested with docker evariste/autodl 
<p>
ALL INFORMATION, SOFTWARE, DOCUMENTATION, AND DATA ARE PROVIDED "AS-IS".
UNIVERSITE PARIS SUD, INRIA, CHALEARN, AND/OR OTHER ORGANIZERS
OR CODE AUTHORS DISCLAIM ANY EXPRESSED OR IMPLIED WARRANTIES.

# Introduction

* We target applications of multi-label image classification. 
* Raw data are provided, in TF record format.  
* We impose restrictions on training time and resources to push the state-of-the-art further.
* This notebook uses sample data. Download larger datasets from the website of the challenge.

In [ ]:
model_dir = 'AutoDL_sample_code_submission/'      # Where the code you should submit is found (called model.py)
simple_dir = 'AutoDL_simple_baseline_models/'    # Where you find some examples of code to replace model.py
lib_dir = 'AutoDL_ingestion_program/'             # Useful libraries used on Codalab
score_dir = 'AutoDL_scoring_program/'             # Scoring program 
from sys import path; path.append(model_dir); path.append(lib_dir); path.append(score_dir); 
%matplotlib inline
# Uncomment the next lines to auto-reload libraries (this causes some problem with pickles in Python 3)
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import warnings; warnings.filterwarnings("ignore")
from os.path import join

<div style="background:#cfdfff">
    <h1> Step 1: Load data </h1>
<p>
We provide sample_data with the starting kit, but you may fetch other public_data from the challenge website and subtitute it to sample_data.
</div>

In [ ]:
data_dir = 'AutoDL_sample_data'            # Change this directory and the dataset as needed
data_name = 'miniciao' 
!ls $data_dir*

In [ ]:
from dataset import AutoDLDataset
D_train = AutoDLDataset(join(data_dir, data_name + '.data', "train"))
D_test = AutoDLDataset(join(data_dir, data_name + '.data', "test"))

<div style="background:#cfdfff">
    <h1> Step 2: Exploratory data analysis </h1>
    <br>
    <font color="ff0000"> THIS NEEDS MORE EXAMPLES OF DATA VISUALIZATION. T-SNE? We also need to show the TEXT labels.</font>
</div>


In [ ]:
D_train.show_image(1)
D_train.show_image(11)

In [ ]:
class_labels=D_train.get_class_labels()
print(class_labels)

<div style="background:#cfdfff">
    <h1>Step 3: Building a predictive model</h1>
    <p>
We provide a simple example of predictive model in the `AutoDL_sample_code_submission/` directory. You can change it to another example by copying other examples from AutoDL_simple_baseline_models.
</div>

In [ ]:
my_chosen_model = join(simple_dir, 'linear.py')  # all_zero.py, linear.py, or 2dcnn.py
my_submitted_model = join(model_dir, 'model.py') # submitted models must be called model.py
!cp $my_chosen_model $my_submitted_model
!rm -r checkpoints*                              # clean up previous models

In [ ]:
from model import Model
??Model
time_budget=100
M = Model(D_train.get_metadata())

In [ ]:
M.train(D_train.get_dataset(),
                  remaining_time_budget=time_budget)

In [ ]:
prediction = M.test(D_test.get_dataset(), remaining_time_budget=time_budget)

<div style="background:#cfdfff">
<h1> Step 4: Compute score </h1> 
Compute the balanced accuracy for the chosen duration. For this challenge, you will be judged on the area under the learning curve, computed for increasing training times.
</div>

In [ ]:
from score import autodl_bac
from libscores import read_array
solution_file = join(data_dir, data_name + '.solution')
solution = read_array(solution_file)
current_bac = autodl_bac(solution, prediction)
print('Number of test examples: %d \n\t\t Solution \t\t\t\t\t Prediction ' % len(solution))
[print(z) for z in zip(solution, prediction)]
print("Normalized balanced accuracy (2*BAC-1) = {:.4f}.".format(current_bac))

<div style="background:#cfdfff">
<h1> Step 5: Making a submission </h1> 

<h2> Unit testing </h2> 

It is <b><span style="color:red">important that you test your submission files before submitting them</span></b>. All you have to do to make a submission is modify the file <code>model.py</code> in the <code>AutoDL_sample_code_submission/</code> directory, then run this test to make sure everything works fine. This is the actual program that will be run on the server to test your submission. 
</div>

In [ ]:
!python run_local_test.py -dataset_dir=$data_dir -code_dir=$model_dir

<div style="background:#cfdfff">
    <h1> Preparing the submission </h1>

Zip the contents of `AutoDL_sample_code_submission/` (without the directory).
<b><span style="color:red">Do NOT zip the data with your submissions</span></b>.
</div>

In [ ]:
import datetime 
from data_io import zipdir
the_date = datetime.datetime.now().strftime("%y-%m-%d-%H-%M")
# sample_code_submission = 'sample_code_submission_' + the_date + '.zip'
sample_code_submission = 'mysubmission.zip'
zipdir(sample_code_submission, model_dir)
print("Submit this file: " + sample_code_submission)